## 요구사항 정의서 초안

- extracted_additional_requirements.json 파일의 요구사항을 extracted_requirements.json에 추가

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
import json
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Hugging Face 로컬 모델 로드
hf_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Embedding chunks"):
        try:
            embedding = hf_model.encode(text)
            embeddings.append(embedding.tolist())  # numpy → list로 변환
        except Exception as e:
            print(f"Embedding error: {e}")
            embeddings.append(None)
    return embeddings

def chunk_requirements(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    chunks = []
    for item in data:
        # 임베딩에 사용되는 text 데이터
        chunk_text = f"""[ID] {item.get('id', '')}
[유형] {item.get('type', '')}
[설명] {item.get('description', '')}
[요구사항 상세] {item.get('detailed_description', '')}
[수용 조건] {item.get('acceptance_criteria', '')}
[모듈] {item.get('module', '')}
[대분류] {item.get('category_large', '')}
[중분류] {item.get('category_medium', '')}
[소분류] {item.get('category_small', '')}
[중요도] {item.get('importance', '')}
[난이도] {item.get('difficulty', '')}
"""     
        # 요구사항 검색 시 사용되는 key값 추가
        chunks.append({
            "id": item.get("id", ""),
            "type": item.get('type', ''),
            "description": item.get('description', ''),
            "detailed_description": item.get('detailed_description', ''),
            "acceptance_criteria": item.get('acceptance_criteria', ''),
            "module": item.get('module', ''),
            "source_pages": item.get('source_pages', ''),
            "raw_text_snippet": item.get('raw_text_snippet', ''),
            "status": item.get('status', ''),
            "mod_reason": item.get('mod_reason', ''),
            "category_large": item.get('category_large', ''),
            "category_medium": item.get('category_medium', ''),
            "category_small": item.get('category_small', ''),
            "difficulty": item.get('difficulty', ''),
            "importance": item.get('importance', ''),
            "text": chunk_text
        })
    return chunks

def embed_and_save_to_faiss(json_file_path, faiss_index_path="requirements_index.faiss", metadata_path="metadata.json"):
    # 1. chunk & 텍스트 추출
    chunks = chunk_requirements(json_file_path)
    texts = [chunk["text"] for chunk in chunks]
    ids = [chunk["id"] for chunk in chunks]

    # 2. 임베딩
    embeddings = get_embeddings(texts)
    valid_embeddings = []
    valid_ids = []

    for emb, id_ in zip(embeddings, ids):
        if emb is not None:
            valid_embeddings.append(emb)
            valid_ids.append(id_)

    # 3. FAISS 인덱스 생성
    dim = len(valid_embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(valid_embeddings).astype('float32'))

    # 4. 저장
    faiss.write_index(index, faiss_index_path)
    with open(metadata_path, "w", encoding="utf-8") as f:
        json.dump(valid_ids, f, ensure_ascii=False, indent=2)

    print(f"✅ 저장 완료: {faiss_index_path}, 메타데이터: {metadata_path}")

# 사용
embed_and_save_to_faiss("./extracted_requirements.json")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding chunks: 100%|██████████| 104/104 [00:10<00:00,  9.93it/s]

✅ 저장 완료: requirements_index.faiss, 메타데이터: metadata.json


### Test

In [4]:
# 쿼리 정의 및 임베딩 생성
query = "JRE 관련 기능"
query_emb = hf_model.encode(query).astype('float32')

# FAISS 인덱스 및 메타데이터(ID 리스트) 로드
index = faiss.read_index("requirements_index.faiss")
metadata = json.load(open("metadata.json", encoding="utf-8"))

# ID → 텍스트 매핑 로드 (chunk_requirements로부터 생성 가능)
# 원본 JSON에서 id와 함께 텍스트를 다시 생성
chunks = chunk_requirements("extracted_requirements.json")
id_to_text = {chunk["id"]: chunk["text"] for chunk in chunks}

# 검색
D, I = index.search(np.array([query_emb]).astype("float32"), k=5)

# 결과 출력
print("🔍 검색 결과:")
for i, idx in enumerate(I[0]):
    req_id = metadata[idx]
    req_text = id_to_text.get(req_id, "(텍스트 없음)")

    print(f"\n🔹 Top {i+1}")
    print(f"ID: {req_id}")
    print(f"거리: {D[0][i]}")
    print("요구사항 내용:")
    print(req_text)
    print("-" * 80)

🔍 검색 결과:

🔹 Top 1
ID: TCON-003
거리: 1.092711329460144
요구사항 내용:
[ID] TCON-003
[유형] 기술적 제약
[설명] 도입되는 솔루션은 JRE 1.7.0을 사용해야 한다.
[요구사항 상세] [요구사항]  
도입되는 솔루션은 JRE 1.7.0을 사용해야 한다.

[대상업무]  
전체 시스템 - 환경 설정 및 배포 관리

[요건처리 상세]  
전체 시스템의 개발 및 운영 환경에서 Java Runtime Environment(JRE) 1.7.0을 사용하도록 설정한다. 이를 위해 다음과 같은 작업을 수행한다:
1. 개발 환경 설정: 모든 개발자의 로컬 개발 환경에 JRE 1.7.0이 설치되어 있는지 확인하고, 필요한 경우 설치를 지원한다. 개발 도구(예: IDE)에서 JRE 1.7.0을 기본 런타임으로 설정한다.
2. 빌드 및 배포 환경 설정: 빌드 서버 및 배포 서버에 JRE 1.7.0이 설치되어 있는지 확인하고, 설치되지 않은 경우 설치를 진행한다. 빌드 스크립트 및 배포 자동화 도구에서 JRE 1.7.0을 사용하도록 설정을 수정한다.
3. 테스트 환경 설정: 테스트 환경에서 JRE 1.7.0을 사용하여 모든 테스트가 수행되도록 설정한다. 테스트 자동화 도구가 JRE 1.7.0을 사용하도록 설정을 조정한다.
4. 문서화: JRE 1.7.0 사용에 대한 설정 및 설치 절차를 문서화하여 모든 팀원이 참조할 수 있도록 한다. 또한, JRE 버전 변경 시의 영향 분석 및 대응 방안을 포함한다.
[수용 조건] 솔루션이 JRE 1.7.0 환경에서 정상적으로 작동해야 한다.
[모듈] 전체 시스템
[대분류] 시스템 관리
[중분류] 환경 설정 및 배포 관리
[소분류] JRE 설정 및 관리
[중요도] 하
[난이도] 하

--------------------------------------------------------------------------------

🔹 Top 2
ID: NFR-020
거리: 1.1473412513732

## 추가 요구사항

In [5]:
def load_requirements(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    else:
        return []

def save_requirements(path, data):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def extract_similar_id(base_file, query):
    query_emb = hf_model.encode(query).astype("float32")
    index = faiss.read_index("requirements_index.faiss")
    metadata = json.load(open("metadata.json", encoding="utf-8"))
    extracted_chunks = chunk_requirements(base_file)
    id_to_text = {ext_chunk["id"]: ext_chunk["text"] for ext_chunk in extracted_chunks}

    D, I = index.search(np.array([query_emb]).astype("float32"), k=1)
    top_idx = I[0][0]
    top_dist = D[0][0]

    existing_id = metadata[top_idx]
    existing_text = id_to_text.get(existing_id, "")
    print(f"\n📄 유사한 기존 요구사항 ID: {existing_id}")
    print(f"📂 유사한 기존 요구사항:\n{existing_text}")

    return existing_id, existing_text

In [6]:
def update_existing_requirement(current_text, updated):
    prompt = f"""
당신은 시스템 분석 전문가이며, 회의록을 바탕으로 **요구사항을 변경 또는 업데이트**하는 전문가입니다.

아래 "기존 요구사항"을 기준으로 "변경된 요구사항"의 내용을 반영해 **최종 업데이트된 요구사항 카드**를 작성하십시오.  
각 항목은 시스템의 설계, 개발, 테스트, 배포, 운영에 실질적으로 사용될 수 있어야 하며, **사람이 읽기 좋은 카드 형식**으로 구성되어야 합니다.

---

## ✅ 출력 형식 예시

[유형] 요구사항 유형: "기능적", "비기능적" 
[설명] 요구사항 요약 (시스템이 수행해야 하는 기능, 조건 등)
[요구사항 상세] 상세 설명
[수용 조건] 요구사항 충족 여부를 판단할 수 있는 구체적 기준 (없으면 "세부 설계 시 정의"로 표기)
[모듈] 관련 시스템 모듈 (예: "인증 모듈", "전체 시스템"). 불분명하면 "미정"
[대분류] / [중분류] /[소분류] 기능 분류 (예: "보안" / "접근 제어" / "권한 등급 설정")
[중요도] 시스템 성공에 대한 영향도 ("상", "중", "하")  
[난이도] 구현 난이도 ("상", "중", "하")

---

## 🔻 입력

### 1. 기존 요구사항
{current_text}

### 2. 변경된 요구사항
{updated}

---

## 🛠 출력 지침

- 출력은 반드시 위와 같은 구조의 **요구사항 카드 형식**으로 작성하십시오.
- 출력은 하나의 요구사항 카드만 작성하십시오.
- 출력 앞뒤에 다른 문장, 마크다운, JSON, 해설을 절대 붙이지 마십시오.
- 누락된 항목이 있다면 빈칸 없이 **"미정"** 으로 채우십시오.
- "요구사항 상세"와 "요건처리 상세" 항목은 중복되지 않도록 구성하십시오.
- 회의록에서 다음 항목은 요구사항으로 간주하지 말고 무시하십시오:
  - 목차, 업체선정 기준, 제안서 작성 요령, 일반 행정적 절차

"""
    return prompt

In [7]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def call_llm(prompt):
    response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 시스템 분석 전문가이며, 요구사항을 업데이트하는 역할입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
    return response.choices[0].message.content.strip()

In [8]:
import re

# 각 필드를 파싱하는 정규식 패턴
patterns = {
    "id": r"\[ID\] (.+)",
    "type": r"\[유형\] (.+)",
    "description": r"\[설명\] (.+)",
    "detailed_description": r"\[요구사항 상세\]([\s\S]+?)\[수용 조건\]",
    "acceptance_criteria": r"\[수용 조건\] (.+)",
    "module": r"\[모듈\] (.+)",
    "category_large": r"\[대분류\] (.+)",
    "category_medium": r"\[중분류\] (.+)",
    "category_small": r"\[소분류\] (.+)",
    "importance": r"\[중요도\] (.+)",
    "difficulty": r"\[난이도\] (.+)"
}

# 항목 추출 함수
def extract_field(text, pattern):
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

In [ ]:
from pprint import pprint

additional_path = "./extracted_additional_requirements.json"
update_results = []
classified_list = []
base_file = "./extracted_requirements.json"
base_requirements = load_requirements(base_file)

for add_chunk in chunk_requirements(additional_path):
    query = add_chunk["text"]
    query_id = add_chunk["id"]
    status = add_chunk["status"]
    print(f"\n{'='*30}")
    print(f"🔍 추가 요구사항 ID: {query_id}")
    print(f"📥 신규 입력 요구사항:\n{query}")
    
    if status == "신규":
        tmp = {
            "acceptance_criteria": add_chunk["acceptance_criteria"],
            "category_large": add_chunk["category_large"],
            "category_medium": add_chunk["category_medium"],
            "category_small": add_chunk["category_small"],
            "description": add_chunk["description"],
            "detailed_description": add_chunk["detailed_description"],
            "difficulty": add_chunk["difficulty"],
            "id": add_chunk["id"],
            "importance": add_chunk["importance"],
            "module": add_chunk["module"],
            "source_page": add_chunk["source_pages"],
            "type": add_chunk["type"],
            "raw_text_snippet": add_chunk["raw_text_snippet"],
            "status": add_chunk["status"],
            "mod_reason": add_chunk["mod_reason"]
        }
        base_requirements.append(tmp)
        print(f"✅ 신규 요구사항으로 등록됨")

    elif status == "변경":
        similar_id, similar_text = extract_similar_id(base_file, query)
        print(f"♻ 기존 요구사항 '{similar_id}'을(를) 갱신합니다.")

        prompt = update_existing_requirement(similar_text, query)
        text_result = call_llm(prompt)

        # JSON 객체 생성
        item = {key: extract_field(text_result, pat) for key, pat in patterns.items()}

        # 기본값 설정
        item["source_pages"] = add_chunk["source_pages"]
        item["raw_text_snippet"] = add_chunk["raw_text_snippet"]
        item["status"] = add_chunk["status"]
        item["mod_reason"] = add_chunk["mod_reason"]
        chunk_text = text_result

        # 최종 리스트에 추가
        chunks = []
        chunks.append({
            "id": similar_id,
            "type": item.get("type", ""),
            "description": item.get("description", ""),
            "detailed_description": item.get("detailed_description", ""),
            "acceptance_criteria": item.get("acceptance_criteria", ""),
            "module": item.get("module", ""),
            "source_pages": item.get("source_pages", ""),
            "raw_text_snippet": item.get("raw_text_snippet", ""),
            "status": item.get("status", ""),
            "mod_reason": item.get("mod_reason", ""),
            "category_large": item.get("category_large", ""),
            "category_medium": item.get("category_medium", ""),
            "category_small": item.get("category_small", ""),
            "difficulty": item.get("difficulty", ""),
            "importance": item.get("importance", ""),
            "text": chunk_text
        })

        for i, req in enumerate(base_requirements):
            print("i: ", i, " req: ", req)
            if req.get("id", "") == similar_id:
                base_requirements[i] = chunks[0]
                pprint(base_requirements[i])

        print(f"✅ 요구사항 '{similar_id}' 갱신 완료")


🔍 추가 요구사항 ID: FUNC-001
📥 신규 입력 요구사항:
[ID] FUNC-001
[유형] 기능적
[설명] 상담 이력을 기반으로 한 만족도 점수 반영
[요구사항 상세] 상담 이력에서 문항별 점수, 상담 횟수, 상담 후 개선요청 이행 여부 등을 통해 가중치를 적용하여 만족도 점수를 반영
[수용 조건] 추천 인력 선정 시 만족도 점수가 정상적으로 반영되어야 함
[모듈] 인사상담 모듈
[대분류] 인사상담
[중분류] 상담 이력
[소분류] 만족도 점수 반영
[중요도] 상
[난이도] 중

✅ 신규 요구사항으로 등록됨

🔍 추가 요구사항 ID: FUNC-002
📥 신규 입력 요구사항:
[ID] FUNC-002
[유형] 기능적
[설명] 직무 적합성 지수 산출
[요구사항 상세] 내부 평가결과와 이직률 데이터를 조합하여 직무 적합성 지수를 산출
[수용 조건] 추천 인력에 직무 적합성 지수가 정상적으로 보여져야 함
[모듈] 직무 적합도 모듈
[대분류] 직무 적합도
[중분류] 직무 적합성 지수
[소분류] 직무 적합성 지수 산출
[중요도] 상
[난이도] 상

✅ 신규 요구사항으로 등록됨

🔍 추가 요구사항 ID: FUNC-003
📥 신규 입력 요구사항:
[ID] FUNC-003
[유형] 기능적
[설명] 인사 발령안 자동 생성 로직 변경
[요구사항 상세] 인사 발령안을 주소, 성별, 인원수 기준에서 우선순위 필터 기준으로 변경
[수용 조건] 우선순위 필터에 따라 인사 발령안이 자동으로 생성되어야 함
[모듈] 인사 발령안 모듈
[대분류] 인사 발령안
[중분류] 자동 생성 로직
[소분류] 우선순위 필터 적용
[중요도] 상
[난이도] 중


📄 유사한 기존 요구사항 ID: FUNC-007
📂 유사한 기존 요구사항:
[ID] FUNC-007
[유형] 기능적
[설명] 인사발령을 위한 데이터 취합, 기초데이터 생성, 불일치 검증, 발령 반영 기능을 구현해야 한다.
[요구사항 상세] [요구사항]  
인사발령을 위한 데이터 취합, 기초데이터 생성, 불일치 검증, 발령 반영 기능을 구현한다

In [9]:
save_requirements('./updated_requirements.json', base_requirements)